### Import Dependencies

In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns
import nltk
import contractions
import unicodedata
import spacy
from sklearn import metrics
import gensim
import keras
from keras.models import Sequential
from keras.layers import Dropout, Activation, Dense , Flatten
from sklearn.preprocessing import LabelEncoder
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam , SGD 
from keras import regularizers
from sklearn import metrics
%matplotlib inline

In [2]:
df_train = pd.read_csv('train-dataset.csv')
df_test = pd.read_csv('hold-out.csv')

In [6]:
df_test.isnull().sum()


comment          9
offensive        0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [7]:
df_test = df_test[~(df_test['comment'].isnull())]

In [8]:
y_train = df_train['offensive']
y_test = df_test['offensive'][:10000]


X_train = df_train['comment']
X_test = df_test['comment'][:10000]

### Feature Engineering with word embeddings

In [124]:
tokenized_train = [nltk.word_tokenize(t) for t in X_train]
tokenized_test = [nltk.word_tokenize(t) for t in X_test]

num_features = 256 

w2v_model = gensim.models.Word2Vec(tokenized_train, size= num_features, window=150,
                                   min_count=10, sample=1e-3, workers=16)

In [64]:
w2v_train_model.wv.word_vec('kiss')

array([ 1.74355954e-01,  1.11989081e-01,  1.07402645e-01, -1.25456393e-01,
        4.53530289e-02,  1.63861319e-01,  7.32331052e-02,  9.11208093e-02,
        7.19136894e-02, -4.49535549e-02, -9.32257399e-02, -1.26669127e-02,
        1.06848091e-01, -1.35491528e-02, -1.17937051e-01, -7.00152367e-02,
        4.11962643e-02, -1.01207547e-01,  4.14986648e-02, -6.12003123e-03,
        5.81789315e-02,  1.82755496e-02,  1.47855654e-02,  9.13466364e-02,
        6.61480501e-02, -1.23175971e-01, -1.20294884e-01,  1.60996452e-01,
       -3.12828866e-04, -1.73115596e-01,  1.73046529e-01,  6.64712116e-02,
       -6.20119162e-02, -5.72533198e-02,  1.57019615e-01,  4.71708477e-02,
       -1.60932645e-01, -1.32428799e-02,  1.75473869e-01,  3.56723182e-02,
       -8.55835155e-02,  3.42778027e-01, -3.13057378e-02, -7.88839906e-03,
        1.56607196e-01,  1.62414208e-01, -3.59676778e-02, -1.44479901e-01,
        1.22481637e-01,  3.64519469e-02,  1.52686924e-01,  6.62702844e-02,
       -7.03699328e-03, -

### generate averaged word vector features from word2vec model

In [72]:
def averaged_word2vec_vectorizer(corpus , model , num_features):
    vocabulary = set (model.wv.index2word)
    
    def average_word_vectors(words, model , vocabulary , num_features):
        feature_vector = np.zeros((num_features) , dtype = 'float64')
        nwords = 0
        
        for word in words : 
            if word in vocabulary:
                nwords += 1 
                feature_vector = np.add(feature_vector , model.wv[word])
        if nwords: 
            feature_vector = np.divide(feature_vector , nwords)
            
        return feature_vector
    features = [average_word_vectors(tokenized_sentence , model, vocabulary , num_features) 
                for tokenized_sentence in corpus]
    return np.array(features)

In [125]:

avg_wv_train_features = averaged_word2vec_vectorizer(corpus= tokenized_train , model = w2v_model, num_features= num_features)

avg_wv_test_features = averaged_word2vec_vectorizer(corpus= tokenized_test , model = w2v_model , num_features= num_features)

### Converting a label vectors to binary class matrix

In [90]:
le = LabelEncoder()

y_tr = le.fit_transform(y_train)
y_dnn_train = keras.utils.to_categorical(y_tr , num_classes= 2)

y_ts = le.fit_transform(y_test)
y_dnn_test = keras.utils.to_categorical(y_ts , num_classes= 2)

In [89]:
print (len(train_vocabulary))

print (len(test_vocabulary))

5816
3734


## Design neural network architecture

In [92]:
model = Sequential()

# Hidden Layers
model.add(Dense(64, activation='relu', input_shape=(512,), kernel_regularizer = regularizers.l2(0.01) ))
                

model.add(Dropout(0.5))

model.add(Dense(64 , activation = 'relu'))

model.add(Dropout(0.5))

model.add(Dense(64 , activation = 'relu'))

model.add(BatchNormalization())

# Output Layer
model.add(Dense(2, activation='softmax'))

In [93]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 64)                32832     
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 64)                4160      
_________________________________________________________________
batch_normalization_4 (Batch (None, 64)                256       
_________________________________________________________________
dense_16 (Dense)             (None, 2)                 130       
Total para

### Configure Model -- batch_size = 200

In [94]:
model.compile(loss = 'categorical_crossentropy' , optimizer= 'adam' , metrics = ['accuracy'])

In [101]:
model.fit(avg_wv_train_features, y_dnn_train, batch_size=200, epochs=200, verbose=1, validation_data=(avg_wv_test_features, y_dnn_test))

Train on 19995 samples, validate on 10000 samples
Epoch 1/200
19995/19995 [==============================] - 0s 18us/step - loss: 0.2105 - acc: 0.9298 - val_loss: 0.2155 - val_acc: 0.9330
Epoch 2/200
19995/19995 [==============================] - 0s 19us/step - loss: 0.2101 - acc: 0.9290 - val_loss: 0.2158 - val_acc: 0.9325
Epoch 3/200
19995/19995 [==============================] - 0s 17us/step - loss: 0.2080 - acc: 0.9295 - val_loss: 0.2149 - val_acc: 0.9326
Epoch 4/200
19995/19995 [==============================] - 0s 18us/step - loss: 0.2097 - acc: 0.9291 - val_loss: 0.2167 - val_acc: 0.9338
Epoch 5/200
19995/19995 [==============================] - 0s 17us/step - loss: 0.2088 - acc: 0.9298 - val_loss: 0.2133 - val_acc: 0.9325
Epoch 6/200
19995/19995 [==============================] - 0s 17us/step - loss: 0.2087 - acc: 0.9302 - val_loss: 0.2109 - val_acc: 0.9317
Epoch 7/200
19995/19995 [==============================] - 0s 20us/step - loss: 0.2087 - acc: 0.9304 - val_loss: 0.2120 - 

19995/19995 [==============================] - 0s 17us/step - loss: 0.2076 - acc: 0.9301 - val_loss: 0.2152 - val_acc: 0.9318
Epoch 60/200
19995/19995 [==============================] - 0s 17us/step - loss: 0.2070 - acc: 0.9305 - val_loss: 0.2168 - val_acc: 0.9340
Epoch 61/200
19995/19995 [==============================] - 0s 18us/step - loss: 0.2106 - acc: 0.9291 - val_loss: 0.2115 - val_acc: 0.9310
Epoch 62/200
19995/19995 [==============================] - 0s 20us/step - loss: 0.2116 - acc: 0.9287 - val_loss: 0.2165 - val_acc: 0.9341
Epoch 63/200
19995/19995 [==============================] - 0s 18us/step - loss: 0.2079 - acc: 0.9293 - val_loss: 0.2153 - val_acc: 0.9315
Epoch 64/200
19995/19995 [==============================] - 0s 21us/step - loss: 0.2079 - acc: 0.9299 - val_loss: 0.2127 - val_acc: 0.9319
Epoch 65/200
19995/19995 [==============================] - 0s 20us/step - loss: 0.2097 - acc: 0.9278 - val_loss: 0.2196 - val_acc: 0.9342
Epoch 66/200
19995/19995 [==============

Epoch 118/200
19995/19995 [==============================] - 0s 23us/step - loss: 0.2104 - acc: 0.9281 - val_loss: 0.2149 - val_acc: 0.9314
Epoch 119/200
19995/19995 [==============================] - 0s 24us/step - loss: 0.2120 - acc: 0.9276 - val_loss: 0.2200 - val_acc: 0.9332
Epoch 120/200
19995/19995 [==============================] - 0s 20us/step - loss: 0.2067 - acc: 0.9304 - val_loss: 0.2143 - val_acc: 0.9339
Epoch 121/200
19995/19995 [==============================] - 0s 20us/step - loss: 0.2089 - acc: 0.9306 - val_loss: 0.2235 - val_acc: 0.9323
Epoch 122/200
19995/19995 [==============================] - 0s 22us/step - loss: 0.2100 - acc: 0.9300 - val_loss: 0.2191 - val_acc: 0.9340
Epoch 123/200
19995/19995 [==============================] - 0s 23us/step - loss: 0.2102 - acc: 0.9278 - val_loss: 0.2121 - val_acc: 0.9342
Epoch 124/200
19995/19995 [==============================] - 0s 24us/step - loss: 0.2116 - acc: 0.9282 - val_loss: 0.2193 - val_acc: 0.9311
Epoch 125/200
19995/

19995/19995 [==============================] - 0s 23us/step - loss: 0.2085 - acc: 0.9291 - val_loss: 0.2124 - val_acc: 0.9322
Epoch 177/200
19995/19995 [==============================] - 0s 23us/step - loss: 0.2076 - acc: 0.9304 - val_loss: 0.2180 - val_acc: 0.9338
Epoch 178/200
19995/19995 [==============================] - 0s 23us/step - loss: 0.2076 - acc: 0.9305 - val_loss: 0.2163 - val_acc: 0.9318
Epoch 179/200
19995/19995 [==============================] - 0s 21us/step - loss: 0.2085 - acc: 0.9288 - val_loss: 0.2212 - val_acc: 0.9330
Epoch 180/200
19995/19995 [==============================] - 0s 22us/step - loss: 0.2068 - acc: 0.9300 - val_loss: 0.2185 - val_acc: 0.9339
Epoch 181/200
19995/19995 [==============================] - 0s 23us/step - loss: 0.2082 - acc: 0.9304 - val_loss: 0.2219 - val_acc: 0.9324
Epoch 182/200
19995/19995 [==============================] - 0s 22us/step - loss: 0.2090 - acc: 0.9289 - val_loss: 0.2149 - val_acc: 0.9291
Epoch 183/200
19995/19995 [=======

In [102]:
y_pred = model.predict_classes(avg_wv_test_features)

prediction = le.inverse_transform(y_pred)

In [107]:
print (metrics.accuracy_score(y_test , prediction))

print (metrics.confusion_matrix(y_test , prediction))

print("F1 score: ", metrics.f1_score(y_test , prediction))

print("Precision score: " , metrics.precision_score(y_test , prediction))

print("Recall score: " , metrics.recall_score(y_test , prediction))

0.9306
[[8930   59]
 [ 635  376]]
F1 score:  0.5200553250345781
Precision score:  0.864367816091954
Recall score:  0.37190900098911966


### Configure Model -- batch_size = 128

In [108]:
model.fit(avg_wv_train_features, y_dnn_train, batch_size=128, epochs=100, verbose=1, validation_data=(avg_wv_test_features, y_dnn_test))

Train on 19995 samples, validate on 10000 samples
Epoch 1/100
19995/19995 [==============================] - 0s 24us/step - loss: 0.2155 - acc: 0.9270 - val_loss: 0.2179 - val_acc: 0.9307
Epoch 2/100
19995/19995 [==============================] - 0s 20us/step - loss: 0.2134 - acc: 0.9294 - val_loss: 0.2208 - val_acc: 0.9318
Epoch 3/100
19995/19995 [==============================] - 0s 19us/step - loss: 0.2171 - acc: 0.9270 - val_loss: 0.2260 - val_acc: 0.9316
Epoch 4/100
19995/19995 [==============================] - 0s 20us/step - loss: 0.2146 - acc: 0.9284 - val_loss: 0.2195 - val_acc: 0.9337
Epoch 5/100
19995/19995 [==============================] - 0s 22us/step - loss: 0.2146 - acc: 0.9269 - val_loss: 0.2173 - val_acc: 0.9278
Epoch 6/100
19995/19995 [==============================] - 0s 21us/step - loss: 0.2156 - acc: 0.9275 - val_loss: 0.2156 - val_acc: 0.9325
Epoch 7/100
19995/19995 [==============================] - 0s 20us/step - loss: 0.2118 - acc: 0.9292 - val_loss: 0.2149 - 

19995/19995 [==============================] - 0s 20us/step - loss: 0.2139 - acc: 0.9277 - val_loss: 0.2296 - val_acc: 0.9345
Epoch 60/100
19995/19995 [==============================] - 0s 19us/step - loss: 0.2121 - acc: 0.9295 - val_loss: 0.2327 - val_acc: 0.9297
Epoch 61/100
19995/19995 [==============================] - 0s 20us/step - loss: 0.2136 - acc: 0.9283 - val_loss: 0.2284 - val_acc: 0.9319
Epoch 62/100
19995/19995 [==============================] - 0s 20us/step - loss: 0.2127 - acc: 0.9281 - val_loss: 0.2300 - val_acc: 0.9328
Epoch 63/100
19995/19995 [==============================] - 0s 24us/step - loss: 0.2145 - acc: 0.9288 - val_loss: 0.2264 - val_acc: 0.9322
Epoch 64/100
19995/19995 [==============================] - 0s 20us/step - loss: 0.2137 - acc: 0.9279 - val_loss: 0.2143 - val_acc: 0.9343
Epoch 65/100
19995/19995 [==============================] - 0s 20us/step - loss: 0.2136 - acc: 0.9283 - val_loss: 0.2280 - val_acc: 0.9341
Epoch 66/100
19995/19995 [==============

In [109]:
y_pred = model.predict_classes(avg_wv_test_features)

prediction = le.inverse_transform(y_pred)

In [110]:
print (metrics.accuracy_score(y_test , prediction))

print (metrics.confusion_matrix(y_test , prediction))

print("F1 score: ", metrics.f1_score(y_test , prediction))

print("Precision score: " , metrics.precision_score(y_test , prediction))

print("Recall score: " , metrics.recall_score(y_test , prediction))

0.9329
[[8917   72]
 [ 599  412]]
F1 score:  0.5511705685618729
Precision score:  0.8512396694214877
Recall score:  0.4075173095944609


### Configure Model -- batch_size = 64

In [111]:
model.fit(avg_wv_train_features, y_dnn_train, batch_size=64, epochs=100, verbose=1, validation_data=(avg_wv_test_features, y_dnn_test))

Train on 19995 samples, validate on 10000 samples
Epoch 1/100
19995/19995 [==============================] - 1s 37us/step - loss: 0.2256 - acc: 0.9247 - val_loss: 0.2441 - val_acc: 0.9302
Epoch 2/100
19995/19995 [==============================] - 1s 36us/step - loss: 0.2222 - acc: 0.9263 - val_loss: 0.2223 - val_acc: 0.9295
Epoch 3/100
19995/19995 [==============================] - 1s 34us/step - loss: 0.2223 - acc: 0.9265 - val_loss: 0.2342 - val_acc: 0.9320
Epoch 4/100
19995/19995 [==============================] - 1s 33us/step - loss: 0.2183 - acc: 0.9292 - val_loss: 0.2384 - val_acc: 0.9332
Epoch 5/100
19995/19995 [==============================] - 1s 33us/step - loss: 0.2217 - acc: 0.9269 - val_loss: 0.2285 - val_acc: 0.9307
Epoch 6/100
19995/19995 [==============================] - 1s 34us/step - loss: 0.2196 - acc: 0.9283 - val_loss: 0.2287 - val_acc: 0.9296
Epoch 7/100
19995/19995 [==============================] - 1s 33us/step - loss: 0.2203 - acc: 0.9283 - val_loss: 0.2464 - 

19995/19995 [==============================] - 1s 34us/step - loss: 0.2172 - acc: 0.9290 - val_loss: 0.2305 - val_acc: 0.9292
Epoch 60/100
19995/19995 [==============================] - 1s 35us/step - loss: 0.2191 - acc: 0.9267 - val_loss: 0.2403 - val_acc: 0.9319
Epoch 61/100
19995/19995 [==============================] - 1s 34us/step - loss: 0.2208 - acc: 0.9268 - val_loss: 0.2263 - val_acc: 0.9311
Epoch 62/100
19995/19995 [==============================] - 1s 35us/step - loss: 0.2222 - acc: 0.9270 - val_loss: 0.2345 - val_acc: 0.9288
Epoch 63/100
19995/19995 [==============================] - 1s 34us/step - loss: 0.2205 - acc: 0.9268 - val_loss: 0.2328 - val_acc: 0.9335
Epoch 64/100
19995/19995 [==============================] - 1s 34us/step - loss: 0.2199 - acc: 0.9268 - val_loss: 0.2345 - val_acc: 0.9322
Epoch 65/100
19995/19995 [==============================] - 1s 34us/step - loss: 0.2201 - acc: 0.9278 - val_loss: 0.2327 - val_acc: 0.9277
Epoch 66/100
19995/19995 [==============

In [112]:
y_pred = model.predict_classes(avg_wv_test_features)

prediction = le.inverse_transform(y_pred)

In [113]:
print (metrics.accuracy_score(y_test , prediction))

print (metrics.confusion_matrix(y_test , prediction))

print("F1 score: ", metrics.f1_score(y_test , prediction))

print("Precision score: " , metrics.precision_score(y_test , prediction))

print("Recall score: " , metrics.recall_score(y_test , prediction))

0.9302
[[8921   68]
 [ 630  381]]
F1 score:  0.5219178082191781
Precision score:  0.8485523385300668
Recall score:  0.3768545994065282


### Design neural network with Activation function "tanh"

In [154]:
model = Sequential()

# Hidden Layers
model.add(Dense(512, activation='tanh', input_shape=(num_features,), kernel_regularizer = regularizers.l2(0.01) ))
                

model.add(Dropout(0.5))

model.add(Dense(512 , activation = 'relu'))

model.add(Dropout(0.5))

model.add(Dense(512 , activation = 'relu'))

model.add(BatchNormalization())

# Output Layer
model.add(Dense(2, activation='softmax'))

In [155]:
model.compile(loss = 'categorical_crossentropy' , optimizer= 'adam' , metrics = ['accuracy'])

In [159]:
model.fit(avg_wv_train_features, y_dnn_train, batch_size=500, epochs=50, verbose=1, validation_data=(avg_wv_test_features, y_dnn_test))

Train on 19995 samples, validate on 10000 samples
Epoch 1/50
19995/19995 [==============================] - 1s 54us/step - loss: 0.2092 - acc: 0.9316 - val_loss: 0.2094 - val_acc: 0.9258
Epoch 2/50
19995/19995 [==============================] - 1s 55us/step - loss: 0.2036 - acc: 0.9315 - val_loss: 0.2007 - val_acc: 0.9327
Epoch 3/50
19995/19995 [==============================] - 1s 53us/step - loss: 0.2057 - acc: 0.9312 - val_loss: 0.2004 - val_acc: 0.9314
Epoch 4/50
19995/19995 [==============================] - 1s 52us/step - loss: 0.2066 - acc: 0.9308 - val_loss: 0.1988 - val_acc: 0.9317
Epoch 5/50
19995/19995 [==============================] - 1s 53us/step - loss: 0.2034 - acc: 0.9304 - val_loss: 0.2074 - val_acc: 0.9325
Epoch 6/50
19995/19995 [==============================] - 1s 55us/step - loss: 0.2060 - acc: 0.9302 - val_loss: 0.2039 - val_acc: 0.9313
Epoch 7/50
19995/19995 [==============================] - 1s 53us/step - loss: 0.2045 - acc: 0.9315 - val_loss: 0.2011 - val_acc

In [160]:
y_pred = model.predict_classes(avg_wv_test_features)

prediction = le.inverse_transform(y_pred)

In [161]:
print (metrics.accuracy_score(y_test , prediction))

print (metrics.confusion_matrix(y_test , prediction))

print("F1 score: ", metrics.f1_score(y_test , prediction))

print("Precision score: " , metrics.precision_score(y_test , prediction))

print("Recall score: " , metrics.recall_score(y_test , prediction))

0.93
[[8934   55]
 [ 645  366]]
F1 score:  0.5111731843575419
Precision score:  0.8693586698337292
Recall score:  0.3620178041543027
